# NLP - Homework 1
### Miguel Bonilla

- [Compare Name](#Compare-Name)
    - [Edit Distance](#Edit-Distance)
    - [String Match](#String-Match)
- [Stop Words](#Stop-Words)
- [Stemmer](#Stemmer)

1. Compare your given name with your nickname (if you don’t have a nickname, invent one for this 
assignment) by answering the following questions:
a. What is the edit distance between your nickname and your given name?
b. What is the percentage string match between your nickname and your given name?
Show your work for both calculations.
2. Find a friend (or family member or classmate) who you know has read a certain book. Without 
your friend knowing, copy the first two sentences of that book. Now rewrite the words from 
those sentences, excluding stop words. Now tell your friend to guess which book the words are 
from by reading them just that list of words. Did you friend correctly guess the book on the first 
try? What did he or she guess? Explain why you think you friend either was or was not able to 
guess the book from hearing the list of words. 
3. Run one of the stemmers available in Python. Run the same two sentences from question 2 
above through the stemmer and show the results. How many of the outputted stems are valid 
morphological roots of the corresponding words? Express this answer as a percentage.

In [1]:
import copy
import numpy as np
import pandas as pd
import nltk
import contractions
from nltk.tokenize import word_tokenize

### Compare Name
#### Edit Distance

Given Name: Miguel - Nickname: Migue 

Setting up the matrix manually:

-|M|i|g|u|e|l
-|-|-|-|-|-|-|
M|0|1|2|3|4|5
i|1|0|1|2|3|4
g|2|1|0|1|2|3
u|3|2|1|0|1|2
e|4|3|2|1|0|1

Distance between Miguel and Migue = 1. Since they are the same with one fewer letter for the nickname.

In [2]:
## Calculating distance using formula from book:
def levenshtein_edit_distance(u, v):
    # convert to lower case
    u = u.lower()
    v = v.lower()
    # base cases
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    # initialize edit distance matrix
    edit_matrix = []
    # initialize two distance matrices
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    # du: the previous row of edit distances
    for i in range(len(du)):
        du[i] = i
    # dv : the current row of edit distances
    for i in range(len(u)):
        dv[0] = i + 1
        # compute cost as per algorithm
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        # assign dv to du for next iteration
        for j in range(len(du)):
            du[j] = dv[j]
        # copy dv to the edit matrix
        edit_matrix.append(copy.copy(dv))
    # compute the final edit distance and edit matrix
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    print("distance = ",distance)
    return edit_matrix

In [3]:
levenshtein_edit_distance('Miguel','Migue')

distance =  1


,m,i,g,u,e,l
m,0,1,2,3,4,5
i,1,0,1,2,3,4
g,2,1,0,1,2,3
u,3,2,1,0,1,2
e,4,3,2,1,0,1


The function given on the book, yields the same results as the hand calculations done above. As expected, given that the name and nickname are exactly the same, with the nickname having one fewer letter.

#### String Match

In [4]:
## will use cosine distance from the book
## first do bag of characters (boc) vectorization
name_list = ['Miguel','Migue']

In [5]:
def string_distance(word_list):
    word_list = [word.lower() for word in word_list]
    unique_chars = np.unique(
                        np.hstack([list(word)
                        for word in word_list]))
    word_list_term_counts = [{char: count
                                  for char, count in np.stack(
                                  np.unique(list(word), return_counts=True), axis=1)}
                                  for word in word_list]
    boc_vectors = [np.array([int(word_term_counts.get(char, 0))
                            for char in unique_chars])
                   for word_term_counts in word_list_term_counts]
    similarity = round((np.dot(boc_vectors[0],boc_vectors[1]) / (np.sqrt(sum(np.square(boc_vectors[0]))) * np.sqrt(sum(np.square(boc_vectors[1]))))),3)
    display(pd.DataFrame(columns=list(unique_chars),data=boc_vectors, index=word_list))
    print("The percentage string match is: {}%".format(similarity*100))
    #return list(unique_chars), boc_vectors

In [6]:
string_distance(name_list)

,e,g,i,l,m,u
miguel,1,1,1,1,1,1
migue,1,1,1,0,1,1


The percentage string match is: 91.3%


### Stop Words

In [7]:
### from Harry Potter and The Sorcerer's Stone
text_string = '''Tell me, Oh Muse, of that ingenious hero who traveled far and wide after he had sacked the famous town of Troy.
Many cities did he visit, and many were the nations with whose manners and customs he was acquainted.'''

In [8]:
stopwords = nltk.corpus.stopwords.words('english')
def text_normalize(text):
    word_tokens = word_tokenize(text)
    content = [w.lower() for w in word_tokens if w.lower() not in stopwords and w.isalpha()]
    return content

In [9]:
content_words = text_normalize(text_string)
print(' '.join(content_words))

tell oh muse ingenious hero traveled far wide sacked famous town troy many cities visit many nations whose manners customs acquainted


#### Results
The friend presented with the text, was able to correctly identify it as being from The Odyssey by the end of the first sentence. They were able to identify the source because the content words retained enough information about the source material to make it identifiable. For example, the words "muse", "hero", "traveled", "sacked", "troy" give enough information on the context, the time, and a journey which make the text identifiable.

### Stemmer

In [10]:
from nltk.stem import PorterStemmer

In [11]:
ps = PorterStemmer()

In [12]:
def ps_stemmer(text):
    word_tokens = word_tokenize(text)
    word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
    for w in word_tokens:
        print(w, " : ", ps.stem(w))

In [13]:
ps_stemmer(','.join(content_words))

tell  :  tell
oh  :  oh
muse  :  muse
ingenious  :  ingeni
hero  :  hero
traveled  :  travel
far  :  far
wide  :  wide
sacked  :  sack
famous  :  famou
town  :  town
troy  :  troy
many  :  mani
cities  :  citi
visit  :  visit
many  :  mani
nations  :  nation
whose  :  whose
manners  :  manner
customs  :  custom
acquainted  :  acquaint


Out of the 20 unique stems that are returned by the function, only 4 (ingeni, famou, mani, and citi) are not valid morphological roots of their corresponding words. In other words 16/20*100 = 80% are valid morphological roots of their associated words.